# Get Tweets into a pandas DataFrame

In [122]:
import pandas as pd
import inflect
import numpy as np

In [75]:
dfTrumpRef = pd.read_json("/Users/maharshi/Desktop/Fall 2019/STOR 565/project/Tweets/tweets_trump.json")
dfBernieRef = pd.read_json("/Users/maharshi/Desktop/Fall 2019/STOR 565/project/Tweets/tweets_bernie.json")

In [76]:
dfTrump = dfTrumpRef.copy()
dfBernie = dfBernieRef.copy()

In [77]:
dfTrump.drop(['html',"is_retweet",'retweet_id','retweeter_userid', 'retweeter_username','timestamp_epochs'], inplace=True, axis=1)
dfBernie.drop(['html',"is_retweet",'retweet_id','retweeter_userid', 'retweeter_username','timestamp_epochs'], inplace=True, axis=1)


In [78]:
dfTrump['affiliation']='republican'
dfBernie['affiliation']='democrat'

In [79]:
df = dfTrump.append(dfBernie, ignore_index = True) 

In [80]:
df

,fullname,likes,replies,retweets,text,timestamp,tweet_id,tweet_url,user_id,username,affiliation
0,Donald J. Trump,23840,6665,7115,The Whistleblower must come forward to explain...,2019-11-01 23:16:07,1190407200109318144,/realDonaldTrump/status/1190407200109318144,25073877,realDonaldTrump,republican
1,Donald J. Trump,66258,18006,14602,You can’t Impeach someone who hasn’t done anyt...,2019-11-01 22:48:59,1190400372180807680,/realDonaldTrump/status/1190400372180807681,25073877,realDonaldTrump,republican
2,Donald J. Trump,90638,14326,18941,"Oh no, Beto just dropped out of race for Presi...",2019-11-01 21:51:45,1190385966621569024,/realDonaldTrump/status/1190385966621569024,25073877,realDonaldTrump,republican
3,Donald J. Trump,47576,7602,11933,Republicans have never been more unified than ...,2019-11-01 21:40:15,1190383074632257536,/realDonaldTrump/status/1190383074632257536,25073877,realDonaldTrump,republican
4,Donald J. Trump,24817,3492,5978,.....with a President and Federal Government t...,2019-11-01 21:11:29,1190375834529468416,/realDonaldTrump/status/1190375834529468423,25073877,realDonaldTrump,republican
5,Donald J. Trump,25409,1092,6322,"....our great police are being disrespected, e...",2019-11-01 21:11:29,1190375831840985088,/realDonaldTrump/status/1190375831840985091,25073877,realDonaldTrump,republican
6,Donald J. Trump,12544,606,3597,"....in New York, and another reason they are l...",2019-11-01 21:11:28,1190375830247104512,/realDonaldTrump/status/1190375830247104513,25073877,realDonaldTrump,republican
7,Donald J. Trump,39836,5355,10469,"I love New York, but New York can never be gre...",2019-11-01 21:11:28,1190375828099674112,/realDonaldTrump/status/1190375828099674113,25073877,realDonaldTrump,republican
8,Donald J. Trump,68609,12141,15862,Stock Market up BIG! Record highs for S&P 500 ...,2019-11-01 14:08:56,1190269494788263936,/realDonaldTrump/status/1190269494788263936,25073877,realDonaldTrump,republican
9,Donald J. Trump,33689,1856,9768,....Our Republican candidate @EddieRispone is ...,2019-11-01 14:08:18,1190269335224430592,/realDonaldTrump/status/1190269335224430592,25073877,realDonaldTrump,republican


In [60]:
tweetList = list(df['text'].values)

# Pre-processing Tweets (get rid of stopwords)

In [42]:
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [111]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(sentance):
    words = nltk.word_tokenize(sentance)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    return words



In [44]:
words = nltk.word_tokenize(tweetList[0])
words = normalize(words)
print(words)

['whistleblower', 'must', 'come', 'forward', 'explain', 'account', 'phone', 'call', 'ukrainian', 'president', 'inaccurate', 'fraudulent', 'whistleblower', 'deal', 'corrupt', 'politician', 'shifty', 'adam', 'schiff', 'andor', 'committee']


In [91]:
' '.join(words)

'whistleblower must come forward explain account phone call ukrainian president inaccurate fraudulent whistleblower deal corrupt politician shifty adam schiff andor committee'

In [117]:
l1 = [normalize(x) for x in tweetList]
l1 = [' '.join(x) for x in l1]
l1 = [' '.join(s for s in x.split() if not any(c.isdigit() for c in s)) for x in l1] # Remove numbers

In [118]:
len(l1)

1613

In [124]:
finaldf = pd.DataFrame({"TweetId":list(df["tweet_id"].values), "CleanText":l1, "party":list(df["affiliation"].values)})

In [125]:
# remove rows with empty fields (CleanText = "")
finaldf['CleanText'].replace('', np.nan, inplace=True)
finaldf.dropna(subset=['CleanText'], inplace=True)

In [126]:
finaldf

,TweetId,CleanText,party
0,1190407200109318144,whistleblower must come forward explain accoun...,republican
1,1190400372180807680,impeach someone done anything wrong,republican
2,1190385966621569024,oh beto dropped race president despite saying ...,republican
3,1190383074632257536,republicans never unified right dems mess corr...,republican
4,1190375834529468416,president federal government wants wonerful ci...,republican
5,1190375831840985088,great police disrespected even water dumped ma...,republican
6,1190375830247104512,new york another reason leaving taxes energy c...,republican
7,1190375828099674112,love new york new york never great current lea...,republican
8,1190269494788263936,stock market big record highs p nasdaq enjoy,republican
9,1190269335224430592,republican candidate eddierispone successful c...,republican


In [127]:
finaldf.to_csv("tweets-20191106-1.csv",index=False)

In [99]:
data

NameError: name 'data' is not defined